In [3]:
#!pip install pyOpenBCI pylsl
#!pip install numpy pyserial bitstring xmltodict requests

In [2]:
# import pyOpenBCI
# from pyOpenBCI import OpenBCICyton

# #For Windows replace '*' with the port number
# #board = OpenBCICyton(port='COM*')

# #For MacOS and Linux replace '*' with the port number
# #board = OpenBCICyton(port='/dev/ttyUSB*')

# #Auto port detection
# board = OpenBCICyton()

In [1]:
# from pyOpenBCI import OpenBCICyton

# def print_raw(sample):
#     print(sample.channels_data)

# board = OpenBCICyton()

# board.start_stream(print_raw)

In [16]:
#Write commands to the board
#board.write_command(command)

import pyOpenBCI
from pyOpenBCI import OpenBCICyton
import numpy as np

#Commands
def print_raw(sample):
    print(sample.channels_data, sample.aux_data)

def return_raw(sample):
    return sample.channels_data, sample.aux_data

def return_filtered(sample):
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)

    return np.array(sample.channels_data) * SCALE_FACTOR_EEG, np.array(sample.aux_data) * SCALE_FACTOR_AUX

def record_data(sample, initial):
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)
    #open a csv file named (initial, _eeg) and write the data to it
    with open(initial + '_eeg.csv', 'a') as file:
        file.write(sample.channels_data * SCALE_FACTOR_EEG, sample.aux_data * SCALE_FACTOR_AUX)

def rec_data_labeled(sample, initial, labels):
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)
    labl = []
    labl.append(labels)
    #open a csv file named (initial, _eeg) and write the data to it
    with open(initial + '_eeg.csv', 'a') as file:
        file.write(np.array(sample.channels_data) * SCALE_FACTOR_EEG, np.array(sample.aux_data) * SCALE_FACTOR_AUX, labl)

def stream_data(comm):
    board = pyOpenBCI.OpenBCICyton()
    board.start_stream(comm)
    return board

#Test use
initial = 'test'
labels = 'test'

board = pyOpenBCI.OpenBCICyton()
board.start_stream(return_filtered)
#board = stream_data(rec_data_labeled(board, initial, labels))

Serial established
Skipped 100 bytes before start found


KeyboardInterrupt: 

In [1]:
#Lets create a tkinter window that connects the headset and displays the data
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pyOpenBCI
from pylsl import StreamInfo, StreamOutlet
import numpy as np

#Create a window
win = tk.Tk()
win.title("OpenBCI GUI")

#Label CogniSync
pgLabel = ttk.Label(win, text="CogniSync")
pgLabel.grid(row=0, column=0)

#Button for connecting to the headset
def startLSL():
    #messagebox.showinfo("Connection", "Connecting to the headset")
    SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
    SCALE_FACTOR_AUX = 0.002 / (2**4)
    print("Creating LSL stream for EEG. \nName: OpenBCIEEG\nID: OpenBCItestEEG\n")
    info_eeg = StreamInfo('OpenBCIEEG', 'EEG', 8, 250, 'float32', 'OpenBCItestEEG')
    print("Creating LSL stream for AUX. \nName: OpenBCIAUX\nID: OpenBCItestEEG\n")
    info_aux = StreamInfo('OpenBCIAUX', 'AUX', 3, 250, 'float32', 'OpenBCItestAUX')

    outlet_eeg = StreamOutlet(info_eeg)
    outlet_aux = StreamOutlet(info_aux)

    def lsl_streamers(sample):
        outlet_eeg.push_sample(np.array(sample.channels_data)*SCALE_FACTOR_EEG)
        outlet_aux.push_sample(np.array(sample.aux_data)*SCALE_FACTOR_AUX)

    board = pyOpenBCI.OpenBCICyton()
    board.start_stream(lsl_streamers)

#Function to retreive the data from our lsl stream
def getData():
    print("Retreiving data from the LSL stream")
    

#Create a button
connectButton = ttk.Button(win, text="Connect", command=startLSL)
connectButton.grid(row=1, column=1)


#Display the window
win.mainloop()

Creating LSL stream for EEG. 
Name: OpenBCIEEG
ID: OpenBCItestEEG

Creating LSL stream for AUX. 
Name: OpenBCIAUX
ID: OpenBCItestEEG



2024-02-05 14:22:21.774 (  24.240s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-02-05 14:22:21.775 (  24.241s) [          160995]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-02-05 14:22:21.776 (  24.241s) [          160995]      netinterfaces.cpp:91    I

Serial established
Skipped 100 bytes before start found


: 

In [36]:
from pyOpenBCI import OpenBCICyton
from pylsl import StreamInfo, StreamOutlet
import numpy as np

SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
SCALE_FACTOR_AUX = 0.002 / (2**4)

print("Creating LSL stream for EEG. \nName: OpenBCIEEG\nID: OpenBCItestEEG\n")

info_eeg = StreamInfo('OpenBCIEEG', 'EEG', 8, 250, 'float32', 'OpenBCItestEEG')

print("Creating LSL stream for AUX. \nName: OpenBCIAUX\nID: OpenBCItestEEG\n")

info_aux = StreamInfo('OpenBCIAUX', 'AUX', 3, 250, 'float32', 'OpenBCItestAUX')

outlet_eeg = StreamOutlet(info_eeg)
outlet_aux = StreamOutlet(info_aux)
file = open('test.csv', 'a')
tempVal = open('tempVal.txt', 'r')
def lsl_streamers(sample):
    tempVal = open('tempVal.txt', 'r')
    lbl = ''
    if tempVal != None:
        lbl = tempVal.read()
    else:
        lbl = 'normal'
    outlet_eeg.push_sample(np.array(sample.channels_data)*SCALE_FACTOR_EEG)
    outlet_aux.push_sample(np.array(sample.aux_data)*SCALE_FACTOR_AUX)
    print(sample.channels_data, sample.aux_data, lbl)
    #file.write(str(np.array(sample.channels_data)) + str(np.array(sample.aux_data)) + str(lbl) + '\n')
    #Lets write it in csv format
    #file.write(str(np.array(sample.channels_data)) + ',' + str(np.array(sample.aux_data)) + ',' + str(lbl) + '\n')
    #[-5696010 -5612774 -5810755 -5096217 -5523528  7835888 -5356176 -2032599],[-112 7552 2464],left arm
    #Fix to make into csv format
    #Create an array of the 8 channels and the 3 aux channels plus the label for writing as a csv
    for datai in sample.channels_data:
        file.write(str(datai*SCALE_FACTOR_EEG) + ',')
    for dataj in sample.aux_data:
        file.write(str(dataj*SCALE_FACTOR_AUX) + ',')
    file.write(str(lbl) + '\n')
    tempVal.close()

board = OpenBCICyton()

board.start_stream(lsl_streamers)
file.close()

Creating LSL stream for EEG. 
Name: OpenBCIEEG
ID: OpenBCItestEEG

Creating LSL stream for AUX. 
Name: OpenBCIAUX
ID: OpenBCItestEEG



2024-02-06 13:22:20.266 (10608.985s) [          1F5C56]         udp_server.cpp:82    WARN| Could not bind multicast responder for 224.0.0.1 to interface 129.81.163.160 (Can't assign requested address)
2024-02-06 13:22:20.266 (10608.985s) [          1F5C56]         udp_server.cpp:82    WARN| Could not bind multicast responder for 224.0.0.1 to interface 2620:10a:3042:8155:40f:9bf8:6d6d:d504 (Can't assign requested address)
2024-02-06 13:22:20.266 (10608.985s) [          1F5C56]         udp_server.cpp:82    WARN| Could not bind multicast responder for 224.0.0.1 to interface 2620:10a:3042:8155:f1f6:9069:1171:d087 (Can't assign requested address)
2024-02-06 13:22:20.266 (10608.985s) [          1F5C56]         udp_server.cpp:82    WARN| Could not bind multicast responder for 224.0.0.1 to interface fe80::3479:1dff:fe34:7ba2%awdl0 (Can't assign requested address)
2024-02-06 13:22:20.266 (10608.985s) [          1F5C56]         udp_server.cpp:82    WARN| Could not bind multicast responder for 22

Serial established
Skipped 100 bytes before start found
[0, 0, 0, 0, 0, 0, 0, 0] [-64, 7568, 2512] 
[-5456316, -5422965, -5586755, -5309494, -5361529, 8249506, -5285376, -3158928] [0, 0, 0] 
[-5504095, -5361837, -5424417, -5271784, -5340789, 8256478, -5351953, -3090117] [0, 0, 0] 
[-5520852, -5317644, -5337315, -5251110, -5333055, 8260359, -5383757, -3072306] [0, 0, 0] 
[-5472666, -5376115, -5481522, -5289361, -5350843, 8253963, -5319352, -3142650] [0, 0, 0] 
[-5447572, -5424512, -5594461, -5311698, -5362626, 8249060, -5282027, -3173756] [0, 0, 0] 
[-5489942, -5374881, -5458397, -5279821, -5344344, 8254893, -5339613, -3111216] [0, 0, 0] 
[-5518297, -5317184, -5334050, -5250299, -5332366, 8260423, -5387092, -3076155] [0, 0, 0] 
[-5477071, -5362111, -5448110, -5281576, -5346941, 8255311, -5333881, -3137859] [-80, 7568, 2512] 
[-5441185, -5423161, -5592494, -5312247, -5362808, 8249006, -5281594, -3184887] [0, 0, 0] 
[-5475254, -5388199, -5492387, -5287434, -5348279, 8253378, -5326507, -31

KeyboardInterrupt: 